In [1]:
import pickle
import numpy as np

In [2]:
p = pickle.load(open('waf/trained_waf_model', 'rb'))

In [3]:
vars(p)

{'memory': None,
 'steps': [('vectorizer',
   TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=0.0,
           ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=True,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('classifier', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
             fit_intercept=True, intercept_scaling=1, max_iter=100,
             multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
             solver='liblinear', tol=0.0001, verbose=0, warm_start=False))]}

In [4]:
vec = p.steps[0][1]
vec

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=0.0,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
clf = p.steps[1][1]
clf

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
print(vec.idf_)

[  9.88191796  13.29416517  13.98731235 ...,  14.39277746  14.39277746
  14.39277746]


In [7]:
print(clf.coef_)

[[  3.86345441e+00   2.97867212e-02   1.67598454e-03 ...,   5.48339628e-06
    5.48339628e-06   5.48339628e-06]]


In [8]:
term_influence = vec.idf_ * clf.coef_
print(term_influence)

[[  3.81783395e+01   3.95989592e-01   2.34425193e-02 ...,   7.89213024e-05
    7.89213024e-05   7.89213024e-05]]


In [9]:
print(np.argpartition(term_influence, 1))

[[81937 92199     2 ..., 97829 97830 97831]]


In [10]:
vec.vocabulary_

{'/': 10522,
 'j': 59431,
 'a': 38945,
 'v': 86105,
 's': 79024,
 'c': 43527,
 'r': 76641,
 'i': 57223,
 'p': 72543,
 't': 81489,
 'm': 65680,
 'o': 70279,
 'n': 67925,
 'e': 48183,
 'y': 92321,
 '.': 9860,
 'x': 90144,
 '\n': 154,
 '/j': 11969,
 'ja': 60022,
 'av': 41076,
 'va': 86839,
 'as': 40902,
 'sc': 80190,
 'cr': 45330,
 'ri': 78053,
 'ip': 58842,
 'pt': 74414,
 't/': 81891,
 '/m': 12101,
 'mo': 67318,
 'on': 71801,
 'ne': 69117,
 'ey': 50667,
 'y.': 92541,
 '.e': 10250,
 'ex': 50609,
 'xe': 91218,
 'e\n': 48184,
 '/ja': 11985,
 'jav': 60068,
 'ava': 41103,
 'vas': 86886,
 'asc': 40942,
 'scr': 80235,
 'cri': 45365,
 'rip': 78100,
 'ipt': 58892,
 'pt/': 74433,
 't/m': 81931,
 '/mo': 12138,
 'mon': 67355,
 'one': 71847,
 'ney': 69175,
 'ey.': 50678,
 'y.e': 92549,
 '.ex': 10263,
 'exe': 50644,
 'xe\n': 91219,
 '&': 4002,
 'h': 55125,
 ' ': 348,
 'u': 84107,
 'z': 94408,
 'l': 63324,
 '$': 3487,
 '(': 5071,
 '4': 21211,
 '+': 7199,
 '9': 30764,
 ')': 5690,
 "'": 4414,
 '&&': 4050

In [11]:
# First, we create a token vocabulary dictionary so that
# we can access tokens by index.
vocab = dict([(v,k) for k,v in vec.vocabulary_.items()])

In [12]:
term_idx = np.argpartition(term_influence, 1)[0][0]

In [13]:
print(vocab[term_idx])

t/s


In [14]:
payload = "<script>alert(1)</script>"

In [15]:
p.predict([payload])[0]

1

In [16]:
p.predict_proba([payload])[0]

array([  1.86163618e-09,   9.99999998e-01])

In [17]:
p.predict_proba([payload + '/' + vocab[term_idx]])[0]

array([  1.83734699e-07,   9.99999816e-01])

In [18]:
p.predict_proba([payload + '/' + vocab[term_idx]*258])[0]

array([ 0.50142443,  0.49857557])

In [19]:
p.predict([payload + '/' + vocab[term_idx]*258])[0]

0

In [20]:
print(payload + '/' + vocab[term_idx]*258)

<script>alert(1)</script>/t/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/st/s
